# best

In [97]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from glob2 import glob  
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from pandasql import sqldf
import random
import os
from numba import jit

test_num = 17

# 경고 끄기
warnings.filterwarnings(action='ignore')

# 시드고정
tf.random.set_seed(19970119)
random.seed(19970119)
np.random.seed(19970119)

&nbsp;

## 입력 shape 및 형태 정의 함수

In [98]:
@jit
def make_Tensor(array):
    return tf.convert_to_tensor(array, dtype=tf.float32)

@jit
def astype_data(data):
    df = data.astype(np.float32)
    return make_Tensor(df)

&nbsp;

## Transformer 정의

- encoder

In [99]:
@jit
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

- build

In [100]:
@jit
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0.2, mlp_dropout=0.2):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(28)(x) # 4주 예측
    return keras.Model(inputs, outputs)

&nbsp;

## keras eraly stop, chekpoint 정의

In [101]:
@jit
def call_back_set(name, epoch, batch_size):
    early_stopping = EarlyStopping(monitor='val_loss', patience=30)

    if os.path.exists(f'./check') == False:
        os.mkdir(f'./check')

    filename = f'./check/{name}-{epoch}-{batch_size}.h5'

    checkpoint = ModelCheckpoint(filename,
                                 monitor='val_loss',
                                 verbose=0,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='auto'
                                 )
    return [early_stopping, checkpoint]

&nbsp;

## Model 훈련 함수

In [102]:
def train(x_train, y_train, x_val, y_val, name, epoch, batch_size, learning_rate = 0.001, verbose = 0):
    model = build_model(
    x_train.shape[1:],
    head_size=256,
    num_heads=3,
    ff_dim=4,
    num_transformer_blocks=3,
    mlp_units=[128],
    mlp_dropout=0.2,
    dropout=0.2,
    )

    model.compile(
        loss="mean_squared_error",
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    )

    # Train the model
    with tf.device('/device:GPU:1'):
        history1 = model.fit(
            x_train, y_train,
            epochs = epoch,
            steps_per_epoch=len(x_train) / batch_size,
            batch_size=batch_size,
            validation_data=(x_val, y_val),
            validation_steps=len(x_val) / batch_size,
            shuffle=False,
            callbacks=call_back_set(name, epoch, batch_size),
            verbose=verbose)

    return model

&nbsp;

## 시점 윈도우 생성 함수

In [103]:
@jit
def x_time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in range(len(df) - seqence_length):
        result.append(df[index: index + seqence_length].values)

    return np.array(result)

@jit
def y_time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in range(14, len(df) - seqence_length):
        result.append(df[index: index + seqence_length].values - df.iloc[index-1])

    return np.array(result)

&nbsp;

## 데이터 불러오기 및 parameter 설정

In [104]:
data_list = glob('./aT_data/data/train/*.csv')
epoch = 1
batch = 64

In [105]:

def feature_engineer(df):
       #시간 코사인 인코딩
       if df['datadate'].all() == 0:
         df['datadate'] = 20180520
       df['datetime'] = pd.to_datetime(df['datadate'], format="%Y%m%d")
       df['cos_time'] = np.cos(2*np.pi*df['datetime'].dt.dayofyear/365)

       # 결측치
       df = df.replace(' ', np.nan)
       df = df.fillna(0)

       # 주산지 초기온도, 최대온도, 최저온도, 평균온도, 강수량, 습도
       df['주산지_초기온도'] = (df['주산지_0_초기온도(℃)'] + df['주산지_1_초기온도(℃)'] + df['주산지_2_초기온도(℃)'])/3
       df['주산지_최대온도'] = (df['주산지_0_최대온도(℃)'] + df['주산지_1_최대온도(℃)'] + df['주산지_2_최대온도(℃)'])/3
       df['주산지_최저온도'] = (df['주산지_0_최저온도(℃)'] + df['주산지_1_최저온도(℃)'] + df['주산지_2_최저온도(℃)'])/3
       df['주산지_평균온도'] = (df['주산지_0_평균온도(℃)'] + df['주산지_1_평균온도(℃)'] + df['주산지_2_평균온도(℃)'])/3
       df['주산지_강수량'] = (df['주산지_0_강수량(ml)'] + df['주산지_1_강수량(ml)'] + df['주산지_2_강수량(ml)'])/3

       # 필요한 컬럼 선택
       market_cols = ['단가(원)', '거래량'] # , '경매건수', '거래대금(원)'

       market_code_cols = [] # '도매시장코드', '도매법인코드', '산지코드 '

       market_statistics_cols = ['해당일자_전체거래물량(kg)', '일자별_도매가격_평균(원)','하위가격 평균가(원)', '상위가격 평균가(원)' ] 
       # '하위가격 거래물량(kg)', '상위가격 거래물량(kg)', '일자별_도매가격_최대(원)', , '일자별_도매가격_최소(원)', '일자별_소매가격_최대(원)', '일자별_소매가격_평균(원)', '일자별_소매가격_최소(원)', 
       
       export_cols = [] # '수출중량(kg)', '수출금액(달러)', '수입중량(kg)', '수입금액(달러)', '무역수지(달러)'
       
       date_cols = ['cos_time'] # ,, '일자구분_중순', '일자구분_초순', '일자구분_하순', '월구분_10월', '월구분_11월', '월구분_12월', '월구분_1월', '월구분_2월','월구분_3월', '월구분_4월', '월구분_5월', '월구분_6월', '월구분_7월', '월구분_8월', '월구분_9월', 'datetime'

       weather_cols = ['주산지_초기온도','주산지_최대온도', '주산지_최저온도', '주산지_평균온도', '주산지_강수량'] 
      #  '주산지_0_초기온도(℃)', '주산지_1_초기온도(℃)', '주산지_2_초기온도(℃)',
      #  '주산지_0_최대온도(℃)', '주산지_1_최대온도(℃)', '주산지_2_최대온도(℃)',
      #  '주산지_0_최저온도(℃)', '주산지_1_최저온도(℃)', '주산지_2_최저온도(℃)',
      #  '주산지_0_평균온도(℃)', '주산지_1_평균온도(℃)', '주산지_2_평균온도(℃)',
      #  '주산지_0_강수량(ml)', '주산지_1_강수량(ml)', '주산지_2_강수량(ml)',
      #  '주산지_0_습도(%)', '주산지_1_습도(%)', '주산지_2_습도(%)'

       selected_columns = ['datadate', '해당일자_전체평균가격(원)']  + market_cols + market_code_cols + market_statistics_cols + export_cols +  date_cols + weather_cols # train 에서 사용하지 않는 열

       df = df[[c for c in df.columns if c in selected_columns]]

       return df

&nbsp;

## Train 과정

In [106]:
from sklearn.preprocessing import MinMaxScaler

losses = []
scalers = {}

for i in tqdm(data_list):
    df_number = i.split("_")[-1].split(".")[0]
    df = pd.read_csv(i)

    # 사용할 열 선택 및 index 설정
    df = feature_engineer(df)
    df.set_index('datadate', drop=True, inplace=True)
    
    # nan 처리
    df = df.replace({' ': np.nan})
    df = df.fillna(0)
    
    # 변수와 타겟 분리
    x, y = df[[i for i in df.columns if i != '해당일자_전체평균가격(원)']], df['해당일자_전체평균가격(원)']
    scaler = MinMaxScaler()
    x = pd.DataFrame(scaler.fit_transform(x))
    scalers[df_number] = scaler

    # 2주 입력을 통한 이후 4주 예측을 위해 y의 첫 14일을 제외
    # y = y[14:]

    # time series window 생성
    data_x = time_window(x, 13, 1)
    data_y = y_time_window(y, 27, 1)

    # y의 길이와 같은 길이로 설정
    xdata = data_x[:len(data_y)]
    ydata = data_y
    
    # train, validation 분리 (8 : 2)
    x_train, x_val, y_train, y_val = train_test_split(xdata, ydata, test_size=0.2, shuffle=False, random_state=119)
 
    # transformer 모델 훈련
    transformer = train(astype_data(x_train), y_train, astype_data(x_val), y_val, f'transformer-{df_number}', epoch,
                        batch, learning_rate = 0.002, verbose = 0)
    
    transformer.load_weights(f'./check/transformer-{df_number}-{epoch}-{batch}.h5')
    loss = transformer.evaluate(astype_data(x_val), y_val, verbose=0)
    print('\nvalid_loss: ', loss)
    losses.append(loss)

    if os.path.exists(f'./model') == False:
        os.mkdir(f'./model')

    # 모델 저장
    transformer.save(f'./model/transformer-{df_number}-{epoch}-{batch}.h5')

  0%|          | 0/37 [00:00<?, ?it/s]2022-09-28 00:23:07.038122: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-28 00:23:07.402337: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
  3%|▎         | 1/37 [00:01<01:10,  1.97s/it]


valid_loss:  1679040.25


2022-09-28 00:23:07.980085: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-28 00:23:08.307669: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
  5%|▌         | 2/37 [00:02<00:46,  1.33s/it]


valid_loss:  819961.4375


2022-09-28 00:23:09.168531: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-28 00:23:09.466176: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
  8%|▊         | 3/37 [00:04<00:42,  1.25s/it]

...


valid_loss:  215519.96875


2022-09-28 00:23:39.698208: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-28 00:23:40.014945: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
 97%|█████████▋| 36/37 [00:34<00:00,  1.13it/s]


valid_loss:  111643520.0


2022-09-28 00:23:40.585461: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-28 00:23:40.911185: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
100%|██████████| 37/37 [00:35<00:00,  1.04it/s]


valid_loss:  710292.125


In [107]:
losses

[1679040.25,
 819961.4375,
 528436.75,
 97453.6328125,
 34038948.0,
 8028599.0,
 5783926.5,
 3380958.5,
 2938472.75,
 71198984.0,
 2357067.75,
 1855499.5,
 1654212.625,
 2695987.25,
 340957.6875,
 744355.1875,
 6792965.5,
 6002071.5,
 6199208.5,
 1031681.4375,
 345607.125,
 1583355.875,
 8670705.0,
 827029.3125,
 270017.8125,
 3531181.0,
 4505874.5,
 14427727.0,
 13615026.0,
 6045929.0,
 1062617.5,
 5623128.5,
 5299667.5,
 473336.5,
 215519.96875,
 111643520.0,
 710292.125]

&nbsp;

## Test 과정

In [108]:
zero_csv = [0 for i in range(14)]  # 시점이 비어있는 데이터 0으로 채우기 위한 변수

for i in tqdm(range(10)):
    data_list = glob(f'./aT_data/data/test/set_{i}/*.csv')
    for idx,j in enumerate(data_list):
        df = pd.read_csv(j)
        
        if len(df) == 0:
            df['zero_non'] = zero_csv
            df = df.fillna(0)
            df.drop('zero_non', axis=1, inplace=True)

        file_number = j.split('test_')[1].split('.')[0]

        # 사용할 열 선택 및 index 설정

        df = feature_engineer(df)
        df = df[[c for c in df.columns if c != '해당일자_전체평균가격(원)']]

        df.set_index('datadate', drop=True, inplace=True)
        
        # # nan 처리
        # df = df.replace({' ': np.nan})
        # df = df.fillna(0)

        # # train input 과 형상 맞추기
        # add_col = [i for i in check_col if i not in df.columns]

        # for a in add_col:
        #     df[a] = 0

        # # ' ' -> nan 으로 변경
        # for a in df.columns:
        #     df[a] = df[a].replace({' ': np.nan})

        # # nan 처리
        # df = df.fillna(0)
        # # df = df.interpolate(method = 'values')

        df = pd.DataFrame(scalers[file_number].transform(df))

        # x_test  생성
        df_test = astype_data(df.values.reshape(1, df.values.shape[0], df.values.shape[1]))

        # model test
        if os.path.exists(f'./model_output/{test_num}') == False:
            os.mkdir(f'./model_output/{test_num}')

        if os.path.exists(f'./model_output/{test_num}/set_{i}') == False:
            os.mkdir(f'./model_output/{test_num}/set_{i}')

        # 해당하는 모델 불러오기
        model_test = tf.keras.models.load_model(f'./model/transformer-{file_number}-{epoch}-{batch}.h5')
        pred = model_test.predict(df_test)


        # 결과 저장
        save_df = pd.DataFrame(pred).T
        save_df.to_csv(f'./model_output/{test_num}/set_{i}/predict_{file_number}.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 38ms/step


2022-09-28 00:23:41.233768: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 33ms/step


2022-09-28 00:23:41.325773: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2022-09-28 00:24:18.762355: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 35ms/step


2022-09-28 00:24:19.155923: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-28 00:24:19.255129: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-28 00:24:19.351219: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 34ms/step


2022-09-28 00:24:19.460224: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-28 00:24:19.552009: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
100%|██████████| 10/10 [00:38<00:00,  3.85s/it]


&nbsp;

## 정답 제출 파일생성

In [109]:
for k in tqdm(range(10)):
  globals()[f'set_df_{k}'] = pd.DataFrame()
  answer_df_list = glob(f'./model_output/{test_num}/set_{k}/*.csv') # 예측한 결과 불러오기
  pum_list = glob(f'./aT_data/aT_test_raw/sep_{k}/*.csv') # 기존 test input 불러오기
  pummok = [a for a in pum_list if 'pummok' in a.split('/')[-1]]

  for i in answer_df_list:
    df = pd.read_csv(i)
    number = i.split('_')[-1].split('.')[0]

    base_number = 0
    for p in pummok:
      if number == p.split('_')[-1].split('.')[0]:
        pum_df = pd.read_csv(p)

        if len(pum_df) != 0:
           base_number = pum_df.iloc[len(pum_df)-1]['해당일자_전체평균가격(원)']  # 기존 각 sep 마다 test input의 마지막 target 값 가져오기 (변동률 계산을 위해)
        else:
          base_number = np.nan

    globals()[f'set_df_{k}'][f'품목{number}']  = [base_number] + list(df[df.columns[-1]].values) # 각 품목당 순서를 t, t+1 ... t+28 로 변경

  globals()[f'set_df_{k}'] = globals()[f'set_df_{k}'][[f'품목{col}' for col in range(37)]] # 열 순서를 품목0 ~ 품목36 으로 변경

100%|██████████| 10/10 [00:01<00:00,  6.90it/s]


- 변동률 계산을 위한 t, t+1 ... t+28 설정

In [110]:
set_df_0

,품목0,품목1,품목2,품목3,품목4,품목5,품목6,품목7,품목8,품목9,...,품목27,품목28,품목29,품목30,품목31,품목32,품목33,품목34,품목35,품목36
0,3871.125000,1362.117613,2909.783785,3400.075583,3947.809169,9253.947514,2717.280000,3361.030923,4911.899864,1173.018633,...,8640.811309,602.005658,1105.412623,1566.274239,3633.464557,5454.710444,5619.188362,5230.620027,2905.100888,2087.675036
1,-0.081870,0.074852,-0.154469,-0.001254,0.135612,-0.107048,0.133038,-0.040790,0.025041,-0.039719,...,-0.171847,-0.079323,0.052147,-0.294156,0.093019,-0.022026,-0.312477,-0.102238,0.032309,-0.000369
2,-0.110311,-0.005178,0.041741,0.095035,0.101359,-0.230833,0.179566,-0.237868,0.079583,-0.048661,...,-0.421356,0.035242,-0.139542,-0.089146,-0.042147,-0.318338,-0.013110,0.041940,-0.008803,-0.153753
3,-0.003078,-0.191846,-0.148452,-0.265654,-0.109891,-0.019821,0.055776,-0.167166,-0.039779,-0.092593,...,-0.161707,0.030426,-0.185304,-0.083325,-0.091512,0.048788,0.074908,-0.171834,-0.277242,0.112325
4,-0.058841,-0.009157,-0.023146,-0.198924,0.103976,-0.069671,-0.060409,0.018535,-0.127289,0.085170,...,0.110693,0.066697,-0.030572,-0.143473,0.143820,0.154136,0.031341,-0.015224,-0.117547,-0.119274
5,-0.047421,-0.032627,0.057941,0.047827,0.003260,0.147030,0.046323,-0.129557,0.057108,-0.042467,...,-0.288010,-0.193166,-0.193611,0.045949,-0.003679,-0.331364,0.173600,-0.017443,-0.147110,-0.194647
6,-0.105600,0.057912,-0.206427,-0.013137,0.040522,-0.060739,0.102488,-0.214844,-0.092035,-0.056977,...,-0.028958,-0.057511,-0.009862,-0.011015,0.057403,-0.133926,-0.039363,0.005048,0.048370,0.065178
7,0.156740,0.027145,-0.198239,-0.153262,0.160374,-0.014653,0.022074,0.043407,0.088578,0.003393,...,-0.290767,0.113928,-0.017917,0.040937,0.058994,-0.161599,-0.013443,-0.137775,-0.082267,-0.015270
8,-0.107123,0.091745,0.080058,-0.111577,0.246185,0.247396,-0.029416,-0.097551,-0.070470,-0.089310,...,-0.119534,-0.117810,-0.002882,-0.035233,0.059844,-0.446125,0.122083,-0.171744,-0.124691,0.094922
9,-0.117043,-0.058097,-0.335975,-0.124769,-0.147788,-0.061768,0.099191,-0.200936,-0.097182,-0.148130,...,-0.045754,-0.094014,-0.111134,-0.580332,-0.069177,-0.112930,0.107484,-0.017990,0.122845,-0.224561


- 변동률 계산 

In [111]:
date = [f'd+{i}' for i in range(1,15)] + ['d+22 ~ 28 평균']

for k in range(10):
  globals()[f'answer_df_{k}'] = pd.DataFrame()
  for c in globals()[f'set_df_{k}'].columns:
    base_d = globals()[f'set_df_{k}'][c][0] # 변동률 기준 t 값
    ans_1_14 = []
    for i in range(14):
      ans_1_14.append((globals()[f'set_df_{k}'][c].iloc[i+1])/base_d)
    
    ans_22_28 = (globals()[f'set_df_{k}'][c][22:29].mean())/base_d
    globals()[f'answer_df_{k}'][f'{c} 변동률'] = ans_1_14 + [ans_22_28]
  
  globals()[f'answer_df_{k}']['Set'] = k # set 번호 설정
  globals()[f'answer_df_{k}']['일자'] = date # 일자 설정

- sep 0  ~ sep 9 까지 합치기

In [112]:
# 위에서 계산된 변동률 들을 합쳐주는 과정

all_df =pd.DataFrame()
for i in range(10):
  if i== 0 :
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']],axis=1)
  else:
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']])


all_df = all_df[['Set','일자'] + list(all_df.columns[:-2])]
all_df.reset_index(drop=True, inplace=True)

In [113]:
answer_df_0

,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,품목8 변동률,품목9 변동률,...,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률,Set,일자
0,-2.114899e-05,5.495246e-05,-0.000053,-3.688581e-07,3.435116e-05,-0.000012,0.000049,-0.000012,0.000005,-0.000034,...,0.000047,-0.000188,0.000026,-0.000004,-0.000056,-1.954601e-05,0.000011,-1.768177e-07,0,d+1
1,-2.849578e-05,-3.801668e-06,0.000014,2.795093e-05,2.567463e-05,-0.000025,0.000066,-0.000071,0.000016,-0.000041,...,-0.000126,-0.000057,-0.000012,-0.000058,-0.000002,8.018089e-06,-0.000003,-7.364773e-05,0,d+2
2,-7.950957e-07,-1.408441e-04,-0.000051,-7.813190e-05,-2.783603e-05,-0.000002,0.000021,-0.000050,-0.000008,-0.000079,...,-0.000168,-0.000053,-0.000025,0.000009,0.000013,-3.285150e-05,-0.000095,5.380373e-05,0,d+3
3,-1.519986e-05,-6.722919e-06,-0.000008,-5.850575e-05,2.633777e-05,-0.000008,-0.000022,0.000006,-0.000026,0.000073,...,-0.000028,-0.000092,0.000040,0.000028,0.000006,-2.910604e-06,-0.000040,-5.713238e-05,0,d+4
4,-1.224994e-05,-2.395351e-05,0.000020,1.406655e-05,8.258353e-07,0.000016,0.000017,-0.000039,0.000012,-0.000036,...,-0.000175,0.000029,-0.000001,-0.000061,0.000031,-3.334854e-06,-0.000051,-9.323612e-05,0,d+5
5,-2.727886e-05,4.251606e-05,-0.000071,-3.863715e-06,1.026431e-05,-0.000007,0.000038,-0.000064,-0.000019,-0.000049,...,-0.000009,-0.000007,0.000016,-0.000025,-0.000007,9.651759e-07,0.000017,3.122038e-05,0,d+6
6,4.048949e-05,1.992855e-05,-0.000068,-4.507620e-05,4.062354e-05,-0.000002,0.000008,0.000013,0.000018,0.000003,...,-0.000016,0.000026,0.000016,-0.000030,-0.000002,-2.634010e-05,-0.000028,-7.314565e-06,0,d+7
7,-2.767242e-05,6.735477e-05,0.000028,-3.281595e-05,6.235980e-05,0.000027,-0.000011,-0.000029,-0.000014,-0.000076,...,-0.000003,-0.000022,0.000016,-0.000082,0.000022,-3.283430e-05,-0.000043,4.546787e-05,0,d+8
8,-3.023492e-05,-4.265164e-05,-0.000115,-3.669599e-05,-3.743551e-05,-0.000007,0.000037,-0.000060,-0.000020,-0.000126,...,-0.000101,-0.000371,-0.000019,-0.000021,0.000019,-3.439296e-06,0.000042,-1.075650e-04,0,d+9
9,-6.694954e-05,-1.178238e-05,-0.000013,-3.776936e-05,-7.652933e-05,0.000017,0.000016,-0.000009,0.000020,-0.000052,...,-0.000032,0.000033,-0.000012,-0.000002,-0.000005,2.642724e-06,-0.000012,-1.340203e-05,0,d+10


- 정답 양식으로 변경

In [114]:
# set, 일자 기억하기위해 따로 저장

re_set = list(all_df['Set'])
re_date = list(all_df['일자'])


# 정답 양식 불러오기
out_ans = pd.read_csv('./aT_data/answer_example.csv')

# 두 dataframe 합치기 (nan + 숫자 = nan 이용)
submit_df = all_df + out_ans

submit_df['Set'] = re_set
submit_df['일자'] = re_date


# 최종 저장
submit_df.to_csv(f'./submit/submit_{test_num}.csv',index=False)

- 계산된 변동률 결과물

In [115]:
all_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,-2.114899e-05,0.000055,-5.308607e-05,-3.688581e-07,3.435116e-05,-0.000012,0.000049,-0.000012,...,-0.000020,-0.000132,0.000047,-0.000188,0.000026,-0.000004,-0.000056,-0.000020,0.000011,-1.768177e-07
1,0,d+2,-2.849578e-05,-0.000004,1.434499e-05,2.795093e-05,2.567463e-05,-0.000025,0.000066,-0.000071,...,-0.000049,0.000059,-0.000126,-0.000057,-0.000012,-0.000058,-0.000002,0.000008,-0.000003,-7.364773e-05
2,0,d+3,-7.950957e-07,-0.000141,-5.101824e-05,-7.813190e-05,-2.783603e-05,-0.000002,0.000021,-0.000050,...,-0.000019,0.000051,-0.000168,-0.000053,-0.000025,0.000009,0.000013,-0.000033,-0.000095,5.380373e-05
3,0,d+4,-1.519986e-05,-0.000007,-7.954503e-06,-5.850575e-05,2.633777e-05,-0.000008,-0.000022,0.000006,...,0.000013,0.000111,-0.000028,-0.000092,0.000040,0.000028,0.000006,-0.000003,-0.000040,-5.713238e-05
4,0,d+5,-1.224994e-05,-0.000024,1.991241e-05,1.406655e-05,8.258353e-07,0.000016,0.000017,-0.000039,...,-0.000033,-0.000321,-0.000175,0.000029,-0.000001,-0.000061,0.000031,-0.000003,-0.000051,-9.323612e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,-2.379852e-06,0.000004,1.296565e-05,-1.082171e-04,-3.334251e-05,-0.000019,0.000024,NaN,...,-0.000026,-0.000248,-0.000067,-0.000195,-0.000059,-0.000022,0.000111,-0.000040,-0.000066,1.023611e-04
146,9,d+12,2.391539e-05,0.000107,-6.063943e-07,2.240952e-05,-3.252162e-05,-0.000018,-0.000001,NaN,...,-0.000043,0.000321,-0.000026,0.000107,0.000002,-0.000049,-0.000049,0.000031,0.000072,-4.710899e-05
147,9,d+13,2.561837e-05,0.000365,3.655130e-04,-6.525877e-05,-9.483037e-05,-0.000005,-0.000022,NaN,...,-0.000022,-0.000029,-0.000122,0.000014,-0.000028,0.000042,-0.000070,-0.000075,-0.000117,9.123462e-05
148,9,d+14,-1.375562e-05,-0.000087,-3.045215e-05,-1.883471e-05,-2.213836e-05,-0.000002,0.000011,NaN,...,-0.000036,0.000284,0.000010,-0.000161,-0.000034,-0.000009,0.000088,-0.000061,-0.000038,1.012043e-04


- 제출 양식

In [116]:
out_ans

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,d+2,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,d+5,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,9,d+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,9,d+13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148,9,d+14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- 제출 양식 반영한 최종 결과물 (**실 제출용**)

In [117]:
submit_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,-0.000021,0.000055,-5.308607e-05,-3.688581e-07,3.435116e-05,-0.000012,0.000049,-0.000012,...,-0.000020,-0.000132,0.000047,-0.000188,0.000026,-0.000004,-0.000056,-0.000020,0.000011,-1.768177e-07
1,0,d+2,-0.000028,-0.000004,1.434499e-05,2.795093e-05,2.567463e-05,-0.000025,NaN,-0.000071,...,-0.000049,0.000059,-0.000126,-0.000057,-0.000012,-0.000058,-0.000002,0.000008,-0.000003,-7.364773e-05
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,0.000021,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,-0.000015,-0.000007,-7.954503e-06,-5.850575e-05,2.633777e-05,-0.000008,NaN,0.000006,...,0.000013,0.000111,-0.000028,-0.000092,0.000040,0.000028,0.000006,-0.000003,-0.000040,-5.713238e-05
4,0,d+5,-0.000012,-0.000024,1.991241e-05,1.406655e-05,8.258353e-07,0.000016,NaN,-0.000039,...,-0.000033,-0.000321,-0.000175,0.000029,-0.000001,-0.000061,0.000031,-0.000003,-0.000051,-9.323612e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,-0.000002,0.000004,1.296565e-05,-1.082171e-04,-3.334251e-05,-0.000019,0.000024,NaN,...,-0.000026,-0.000248,-0.000067,-0.000195,-0.000059,-0.000022,0.000111,-0.000040,-0.000066,1.023611e-04
146,9,d+12,0.000024,0.000107,-6.063943e-07,2.240952e-05,-3.252162e-05,-0.000018,-0.000001,NaN,...,-0.000043,0.000321,-0.000026,0.000107,0.000002,-0.000049,-0.000049,0.000031,0.000072,-4.710899e-05
147,9,d+13,0.000026,0.000365,3.655130e-04,-6.525877e-05,-9.483037e-05,-0.000005,-0.000022,NaN,...,-0.000022,-0.000029,-0.000122,0.000014,-0.000028,0.000042,-0.000070,-0.000075,-0.000117,9.123462e-05
148,9,d+14,-0.000014,-0.000087,-3.045215e-05,-1.883471e-05,-2.213836e-05,-0.000002,0.000011,NaN,...,-0.000036,0.000284,0.000010,-0.000161,-0.000034,-0.000009,0.000088,-0.000061,-0.000038,1.012043e-04
